## İki farklı manyetik alan değerine sahip mıknatısların büyüklüklerine göre elektromanyetik kuvveti hesaplayan formül:



In [ ]:
# Kaynak: https://www.cy-magnetics.com/PullForce-DiscMagnets.htm  // aynı manyetik alan değeri için hesaplama yapılıyor.

import math

# Integrasyon adım sayısı (global bir değişken)
N_disk = 30

###############################################################################
# SIMPSON INTEGRASYONU
###############################################################################


def func_integral_simpson_disk(lo, hi, func):
    """
    [lo, hi] aralığında func fonksiyonunu Simpson kuralıyla yaklaşık olarak
    entegre eden yardımcı fonksiyon.
    """
    global N_disk
    step = (hi - lo) / N_disk

    # "Tam" adımlardaki toplam (f(x_1) + f(x_2) + ... + f(x_{N-1}))
    result1 = 0.0
    x = lo
    for i in range(1, N_disk):
        x += step
        result1 += func(x)
    result1 *= 2  # f(x_1), f(x_2) ... f(x_{N-1}) terimlerini 2 ile çarparız.

    # Yarım adımlardaki toplam (f(x_0 + step/2), f(x_1 + step/2), ...)
    result2 = 0.0
    x = lo + step / 2
    for i in range(N_disk):
        result2 += func(x)
        x += step
    result2 *= 4  # Bu terimleri de 4 ile çarparız.

    # Simpson kuralının final
    result = result1 + result2 + func(lo) + func(hi)
    result *= step / 6
    return result


###############################################################################
# TEK BİR HALKA (RING) ENTEGRASYONU
###############################################################################
def ring_magnet_seta(seta, Z0, Z, R1, R2):
    """
    Orijinal koddaki halka integrasyonunun alt fonksiyonu.
    Burada R1, L1'i temsil eden mıknatısın 'iç' entegre edilen yarıçapı gibi düşünebiliriz.
    R2 ise dışarıda sabit konumlanan diğer parametre gibi...
    (Formül CY magnetics'ten aynen alınıyor.)
    """
    x1 = R2 / math.sqrt(2)
    y1 = R2 / math.sqrt(2)
    K1 = ((x1 - R1 * math.cos(seta))**2 +
          (y1 - R1 * math.sin(seta))**2 +
          (Z - Z0)**2)
    K = math.pow(K1, 1.5)
    D = (R1 * (Z - Z0) * math.sin(seta)) / K
    return D


def integral_first_B(Z0, Z, R1, R2):
    """
    ring_magnet_seta fonksiyonunu 0..2*pi aralığında entegre ediyor.
    """
    return func_integral_simpson_disk(
        0,
        2 * math.pi,
        lambda seta: ring_magnet_seta(seta, Z0, Z, R1, R2)
    )


###############################################################################
# TEK MIKNATISIN YARATTIĞI ALAN BILEŞENI
###############################################################################
def integral_second_B(Z, R1, R2, Br1, L1):
    """
    Asıl fark: Tek bir mıknatısın (Br1) manyetik alanını hesaplıyoruz.
    Orijinalde parametre "Br" idi.
    """
    return (Br1 / (4 * math.pi)) * func_integral_simpson_disk(
        0,
        L1,
        lambda Z0: integral_first_B(Z0, Z, R1, R2)
    )


###############################################################################
# IKI MIKNATISIN KUVVET HESABI (Burada asıl fark: Br2’yi de devreye sokuyoruz)
###############################################################################
def DFZ(Z, R2, Br1, Br2, R1, L1):
    """
    Bu fonksiyon, yukarıda integral_second_B ile hesaplanan By'ı alıyor.
    Sonra "ikinci mıknatıs"ın Br2'sini kullanarak kuvvet değerini elde ediyor.
    """
    # 1) Birinci mıknatısın alanı (Br1) -> By:
    By = integral_second_B(Z, R1, R2, Br1, L1)

    # 2) Orijinal formüldeki çarpanlar (kodun geri kalanı aynı kalıyor).
    product = -2 * math.sqrt(2) * math.pi * By * R2 / 1000

    # 3) İkinci mıknatısın Br2'si kuvvete dahil oluyor.
    #    Orijinalde: dF = Br / (4 * pi) * (1e7) * product
    #    Şimdi:     dF = Br2 / (4 * pi) * (1e7) * product
    dF = (Br2 / (4 * math.pi)) * math.pow(10, 7) * product
    return dF


def integral_third_F_2disk(L1, L, L2, R1, R2, Br1, Br2):
    """
    İki disk mıknatısın arasındaki Z aralığında (L1+L) .. (L1+L+L2)
    Simpson integrasyonunu yapar.
    """
    return func_integral_simpson_disk(
        L1 + L,
        L1 + L + L2,
        lambda Z: DFZ(Z, R2, Br1, Br2, R1, L1)
    ) / 1000


###############################################################################
# ORIJINALDEKI EK DÜZELTME FONKSIYONLARI
###############################################################################
def first_modify(jieguo, R1, L1):
    x = R1 / L1
    a = 0.05171
    b = 0.04518
    c = 0.9728
    M = jieguo * (a * x + b / x + c)
    return M


###############################################################################
# ANA KUVVET HESAPLAMA FONKSIYONU
###############################################################################
def calculate_force_twoBr(R1, R2, L1, L2, L, Br1, Br2, pull_or_push):
    """
    R1, L1 -> 1. mıknatısın yarıçapı ve kalınlığı
    R2, L2 -> 2. mıknatısın yarıçapı ve kalınlığı
    L      -> Mıknatıslar arası mesafe
    Br1    -> Birinci mıknatısın remanans değeri (Tesla)
    Br2    -> İkinci mıknatısın remanans değeri (Tesla)
    pull_or_push -> 1 ise çekme (mutlak değer), 0 ise itme (negatif yönde)
    """
    global N_disk

    # Özel durum: eğer "sıfır mesafe" ve "aynı R, aynı L" ise orijinalde N_disk=10 ve ek düzeltme yapılıyordu.
    if L == 0 and R1 == R2 and L1 == L2:
        N_disk = 10
        result = integral_third_F_2disk(L1, L, L2, R1, R2, Br1, Br2)
        result = first_modify(result, R1, L1)
        result *= 0.83  # orijinalde sabit bir düzeltme çarpanı
    else:
        N_disk = 30
        result = integral_third_F_2disk(L1, L, L2, R1, R2, Br1, Br2)

    # Kuvvet işareti
    # pull_or_push = 1 ise => pozitif (çekme) => mutlak değer al
    # pull_or_push = 0 ise => itme => negatif
    if pull_or_push:
        result = abs(result)
    else:
        result = -abs(result)

    return round(result, 2)


###############################################################################
# ÖRNEK KULLANIM
###############################################################################
if __name__ == "__main__":
    # Örnek değerler
    R1 = 20   # 1. mıknatıs yarıçapı (mm)
    L1 = 10  # 1. mıknatıs kalınlığı (mm)
    R2 = 20   # 2. mıknatıs yarıçapı (mm)
    L2 = 10  # 2. mıknatıs kalınlığı (mm)
    L = 2   # Aradaki mesafe (mm)

    Br1 = 1.158
    Br2 = 1.48

    # pull_or_push = 1 => çekme yönü
    # pull_or_push = 0 => itme yönü
    pull_or_push = 1

    force = calculate_force_twoBr(R1, R2, L1, L2, L, Br1, Br2, pull_or_push)
    print("Calculated Force (two different Br):", force)

Calculated Force (two different Br): 216.14
